In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm

#Visualizaiton imports
import seaborn as sns
import matplotlib.pyplot as plt

#Importing Sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Import Market Basket Models
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

%matplotlib inline

In [133]:
order_products__train = pd.read_csv("Desktop/order_products__train.csv")
order_products__prior = pd.read_csv("Desktop//order_products__prior.csv")
orders = pd.read_csv("Desktop/orders.csv")
products = pd.read_csv("Desktop/products.csv")
aisles = pd.read_csv("Desktop/aisles.csv")
departments = pd.read_csv("Desktop/departments.csv")

In [10]:
for name in filenames:
    print(name, eval(name).shape,": Null values?", eval(name).isnull().any()[0])

aisles (134, 2) : Null values? False
departments (21, 2) : Null values? False
order_products__prior (32434489, 4) : Null values? False
order_products__train (1384617, 4) : Null values? False
orders (3421083, 7) : Null values? False
products (49688, 4) : Null values? False


In [11]:
aisles.head()
departments.head()
#combine aisle and departments
order_products__prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [12]:
opt = order_products__train.merge(products,how='left', on='product_id')
opt = opt.merge(departments,how='left', on='department_id')
opt = opt.merge(aisles,how='left', on='aisle_id')

In [13]:
#converting the id to string to preserve order.
opt['aisle_id']= opt['aisle_id'].astype(str)

In [14]:
#checking the top product aisles.  
opt[['aisle','aisle_id']]['aisle'].value_counts().head()

fresh vegetables              150609
fresh fruits                  150473
packaged vegetables fruits     78493
yogurt                         55240
packaged cheese                41699
Name: aisle, dtype: int64

Preprocessing for Market Basket

In [15]:
#instanciate Oone hot encoder ot break out the data my aisle.
ohe = OneHotEncoder(handle_unknown='ignore')

In [16]:
#test to  see # of aisles
print('# of unique aisles =',len(opt['aisle_id'].unique()))
aisle = ohe.fit_transform(opt['aisle_id'].astype(str).values.reshape(-1,1))

# of unique aisles = 134


In [17]:
#one hot encode the aisles and create a dense matrix.
a_ohe = pd.DataFrame(aisle.toarray())
aisle = a_ohe.to_dense()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: DataFrame/Series.to_dense is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
#add order_id to the one hot encoded matrix
basket = opt[['order_id']].join(aisle)

In [19]:
opt[opt['aisle_id'] == 134]


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle


In [20]:
range(len(opt['aisle_id'].unique()))


range(0, 134)

In [21]:
opt[opt['aisle_id'] == '1']['aisle'].value_counts()


prepared soups salads    2936
Name: aisle, dtype: int64

In [22]:
basket.head()


,order_id,0,1,2,3,4,5,6,7,8,...,124,125,126,127,128,129,130,131,132,133
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [23]:
#return the total items in the basket from the aisles
basket = basket.groupby('order_id').sum()

In [24]:
#covert any values to be a binary. 
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [25]:
aisle_dict = dict(zip(opt['aisle_id'].unique().astype('Int64'),opt['aisle'].unique()))

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  """Entry point for launching an IPython kernel.


In [26]:
basket_sets.columns = list(range(1,len(aisle_dict)+1))
basket_sets.rename(columns=aisle_dict,inplace=True)

In [27]:
basket_sets.head()


,prepared soups salads,specialty cheeses,energy granola bars,instant foods,marinades meat preparation,other,packaged meat,bakery desserts,pasta sauce,kitchen supplies,...,trail mix snack mix,feminine care,body lotions soap,tortillas flat bread,frozen appetizers sides,hot cereal pancake mixes,dry pasta,beauty,muscles joints pain relief,specialty wines champagnes
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
98,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,1,1


In [28]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)


Market Basket Results

In [53]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)


In [54]:
rules.head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Vegetarian Organic Golden Corn),(Organic Mint),0.010389,0.001865,0.000533,0.051282,27.501832,0.000513,1.052089
1,(Organic Mint),(Vegetarian Organic Golden Corn),0.001865,0.010389,0.000533,0.285714,27.501832,0.000513,1.385456
2,(Zero Calorie Lemon Lime Twist Soda),(Organic Avocado),0.005860,0.002397,0.000533,0.090909,37.919192,0.000519,1.097363
3,(Organic Avocado),(Zero Calorie Lemon Lime Twist Soda),0.002397,0.005860,0.000533,0.222222,37.919192,0.000519,1.278179
4,(Strawberries),(Coconut Almond Creamer Blend),0.000533,0.000799,0.000533,1.000000,1251.333333,0.000532,inf


In the above example, the {Diaper} is the antecedent and the {Beer} is the consequent. Both antecedents and consequents can have multiple items. In other words, {Diaper, Gum} -> {Beer, Chips} is a valid rule.

Support is the relative frequency that the rules show up. In many instances, you may want to look for high support in order to make sure it is a useful relationship. However, there may be instances where a low support is useful if you are trying to find “hidden” relationships.

Confidence is a measure of the reliability of the rule. A confidence of .5 in the above example would mean that in 50% of the cases where Diaper and Gum were purchased, the purchase also included Beer and Chips. For product recommendation, a 50% confidence may be perfectly acceptable but in a medical situation, this level may not be high enough.

Lift is the ratio of the observed support to that expected if the two rules were independent (see wikipedia). The basic rule of thumb is that a lift value close to 1 means the rules were completely independent. Lift values > 1 are generally more “interesting” and could be indicative of a useful rule pattern.

In [55]:
#sorts to see the lift greater than or equal to 1
rules[rules['lift'] >= 1.5].head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Vegetarian Organic Golden Corn),(Organic Mint),0.010389,0.001865,0.000533,0.051282,27.501832,0.000513,1.052089
1,(Organic Mint),(Vegetarian Organic Golden Corn),0.001865,0.010389,0.000533,0.285714,27.501832,0.000513,1.385456
2,(Zero Calorie Lemon Lime Twist Soda),(Organic Avocado),0.005860,0.002397,0.000533,0.090909,37.919192,0.000519,1.097363
3,(Organic Avocado),(Zero Calorie Lemon Lime Twist Soda),0.002397,0.005860,0.000533,0.222222,37.919192,0.000519,1.278179
4,(Strawberries),(Coconut Almond Creamer Blend),0.000533,0.000799,0.000533,1.000000,1251.333333,0.000532,inf


Market Basket for Products - Tasks

Need to filter out products that have been reordered to reduce the size of the product matrix. Once I have the reordered products, I can join the one hot encoded items to the order Id's

In [32]:
reorders = opt[opt['reordered'] == 1]


In [33]:
reorders['product_id'] = reorders['product_id'].astype('int64')


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
reorders.head()
reorders.shape

(828824, 9)

In [35]:
reorders.dtypes


order_id              int64
product_id            int64
add_to_cart_order     int64
reordered             int64
product_name         object
aisle_id             object
department_id         int64
department           object
aisle                object
dtype: object

In [36]:
# get list of hi volume products (products that occurr more than 1 times)
hivol = reorders.copy()['product_id'].value_counts().sort_values(ascending=False)\
    [reorders.copy()['product_id'].value_counts().sort_values(ascending=False) > 1].index.tolist()

In [37]:
# mask the reorders dataframe to only include the high vol prods
reorders = reorders[reorders['product_id'].isin(hivol)]

In [75]:
#reorders['hi_dem'] = 
reorders['hi_dem'] = (reorders.copy()['product_id'].value_counts().sort_values(ascending=False)>1)

In [76]:
hidem_ord = reorders[reorders['hi_dem'] == True]


In [77]:
hidem_ord['product_id'] = hidem_ord.copy()['product_id'].astype(str)


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
#tranposes the data to ensure that it is correctly fitted.
product = ohe.fit_transform(hidem_ord['product_id'].values.reshape(-1,1))

In [79]:
p_ohe = pd.DataFrame(product.toarray())
products = p_ohe.to_dense()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: DataFrame/Series.to_dense is deprecated and will be removed in a future version
  


In [80]:
products.shape


(13087, 4823)

In [81]:
basket = hidem_ord[['order_id']].join(products)


In [82]:
#return the total items in the basket from the aisles
basket = basket.groupby('order_id').sum()

In [83]:
#covert any values to be a binary.
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [84]:
pro_dict = dict(zip(hidem_ord['product_id'].unique().astype('Int64'),hidem_ord['product_name'].unique()))

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  """Entry point for launching an IPython kernel.


In [85]:
len(pro_dict)
pro_dict = dict(zip(list(range(0,len(hidem_ord['product_id'].unique()))),hidem_ord['product_name'].unique()))

In [86]:
#rename the columns for easier analysis in rules. 
basket_sets.columns = list(range(0,len(pro_dict)))
basket_sets.rename(columns=pro_dict,inplace=True)

In [87]:
basket_sets.sum().head()


Organic 4% Milk Fat Whole Milk Cottage Cheese    0
Lightly Smoked Sardines in Olive Oil             0
Spring Water                                     2
Cage Free Extra Large Grade AA Eggs              2
Roasted Turkey                                   0
dtype: int64

In [88]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.0005, use_colnames=True)

In [89]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules[['antecedents','consequents','confidence','lift']]

Market Basket for Products

In [60]:
rules[(rules['lift'] >= 100) & (rules['confidence']>.5)]


,antecedents,consequents,confidence,lift
4,(Strawberries),(Coconut Almond Creamer Blend),1.000000,1251.333333
5,(Coconut Almond Creamer Blend),(Strawberries),0.666667,1251.333333
49,(Sliced Mushrooms),(Organic Long Grain Brown Rice),1.000000,268.142857
98,(Dryer Sheet Free & Gentle Fabric Softener),"(Organic Milk Reduced Fat, 2% Milkfat)",0.666667,357.523810
102,(Organic YoKids Smoothie Strawberry Banana Low...,(Organic Lasagna with Meat Sauce),1.000000,220.823529
104,(Hass Avocado Variety),(Thick & Chunky Salsa),1.000000,170.636364
125,(Natural Care Baby Wipes Fragrance Free),(Original Enriched Parboiled Long Grain Rice),1.000000,150.160000
142,(Organic Ezekiel 4:9 Sesame Bread),(Cuties Non Dairy Vanilla Frozen Dessert Sandw...,0.666667,417.111111
157,(Roasted Seaweed Snacks),(Organic Cotton Panty Liners),0.666667,312.833333
158,(Made Good Chocolate Chip Granola Bars),(Organic Lasagna with Meat Sauce),1.000000,220.823529


In [100]:
rules.sort_values(['lift','confidence'],ascending=False).head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
8,(4041),(1751),0.004561,0.003110,0.000829,0.181818,58.460606,0.000815,1.218421
9,(1751),(4041),0.003110,0.004561,0.000829,0.266667,58.460606,0.000815,1.357416
6,(1288),(1609),0.006428,0.008294,0.000829,0.129032,15.558065,0.000776,1.138626
7,(1609),(1288),0.008294,0.006428,0.000829,0.100000,15.558065,0.000776,1.103969
4,(4041),(342),0.004561,0.012648,0.000829,0.181818,14.375559,0.000772,1.206764


In [103]:
hidem_ord['product_id'] = hidem_ord.copy()['product_id'].astype(str)


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [104]:
#tranposes the data to ensure that it is correctly fitted.
product = ohe.fit_transform(hidem_ord['product_id'].values.reshape(-1,1))

In [105]:
p_ohe = pd.DataFrame(product.toarray())
products = p_ohe.to_dense()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: DataFrame/Series.to_dense is deprecated and will be removed in a future version
  


In [106]:
basket = hidem_ord[['product_id']].join(products)

In [107]:
#return the total items in the basket from the aisles
basket = basket.groupby('product_id').sum()

In [108]:
#covert any values to be a binary.
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [109]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.0007, use_colnames=True)

In [112]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules[['antecedents','consequents','confidence','lift']]

In [113]:
rules.sort_values(['lift','confidence'],ascending=False).head()


,antecedents,consequents,confidence,lift
8,(4041),(1751),0.181818,58.460606
9,(1751),(4041),0.266667,58.460606
6,(1288),(1609),0.129032,15.558065
7,(1609),(1288),0.100000,15.558065
4,(4041),(342),0.181818,14.375559


In [114]:
#rename the columns for easier analysis in rules. 
basket_sets.columns = list(range(0,len(pro_dict)))
basket_sets.rename(columns=pro_dict,inplace=True)

In [115]:
user_orders = reorders.merge(orders)


In [116]:
hidem_ord = user_orders[user_orders['hi_dem'] == True]


In [117]:
hidem_ord['product_id'] = hidem_ord.copy()['product_id'].astype(str)


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [118]:
#tranposes the data to ensure that it is correctly fitted.
product = ohe.fit_transform(hidem_ord['product_id'].values.reshape(-1,1))

In [119]:
p_ohe = pd.DataFrame(product.toarray())
products = p_ohe.to_dense()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: DataFrame/Series.to_dense is deprecated and will be removed in a future version
  


In [120]:
basket = hidem_ord[['user_id']].join(products)


In [121]:
#return the total items in the basket from the aisles
basket = basket.groupby('user_id').sum()

In [122]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [132]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.0007, use_colnames=True)

In [125]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules[['antecedents','consequents','confidence','lift']]

In [127]:
rules.sort_values(['lift','confidence'],ascending=True).head()


,antecedents,consequents,confidence,lift
36,(1609),(1199),0.025000,1.400746
37,(1199),(1609),0.044776,1.400746
29,(342),(4057),0.030303,2.321583
28,(4057),(342),0.061224,2.321583
76,(1609),(1942),0.025000,2.843939
